###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [2]:
import pandas as pd

###Archivo meew-mguv.csv

Contiene información del delito Amenazas del 01 de enero del año 2010 al 30 de abril del año 2024.

In [3]:
df = pd.read_csv("https://www.datos.gov.co/resource/meew-mguv.csv?$limit=500000")
df.head()

<ipython-input-3-483ae24b7389>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/meew-mguv.csv?$limit=500000")


,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,cantidad
0,ANTIOQUIA,MEDELLÍN (CT),5001000,ARMA BLANCA / CORTOPUNZANTE,01/01/2010,MASCULINO,ADULTOS,1
1,BOLÍVAR,CARTAGENA (CT),13001000,ARMA BLANCA / CORTOPUNZANTE,01/01/2010,MASCULINO,ADULTOS,2
2,HUILA,PITALITO,41551000,ARMA BLANCA / CORTOPUNZANTE,01/01/2010,MASCULINO,ADULTOS,1
3,NARIÑO,PASTO (CT),52001000,ARMA BLANCA / CORTOPUNZANTE,01/01/2010,FEMENINO,ADULTOS,1
4,BOLÍVAR,CARTAGENA (CT),13001000,ARMA DE FUEGO,01/01/2010,MASCULINO,ADULTOS,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412615 entries, 0 to 412614
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   departamento  412615 non-null  object
 1   municipio     412615 non-null  object
 2   codigo_dane   412615 non-null  object
 3   armas_medios  412615 non-null  object
 4   fecha_hecho   412615 non-null  object
 5   genero        412615 non-null  object
 6   grupo_etario  411333 non-null  object
 7   cantidad      412615 non-null  int64 
dtypes: int64(1), object(7)
memory usage: 25.2+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [5]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad']
threat_crimes = df[relevant_cols]
threat_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412615 entries, 0 to 412614
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   codigo_dane   412615 non-null  object
 1   departamento  412615 non-null  object
 2   municipio     412615 non-null  object
 3   fecha_hecho   412615 non-null  object
 4   genero        412615 non-null  object
 5   grupo_etario  411333 non-null  object
 6   cantidad      412615 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 22.0+ MB


### Convertir columna fecha_hecho a tipo date

In [6]:
# Hacer una copia explícita del DataFrame
threat_crimes_ = threat_crimes.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
threat_crimes_['fecha_hecho'] = pd.to_datetime(threat_crimes_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
threat_crimes_.loc[:, 'fecha_hecho'] = threat_crimes_['fecha_hecho'].dt.year

### Verificar valores nulos

In [7]:
threat_crimes_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,11
genero,0
grupo_etario,1282
cantidad,0


  - Solucionando nulos de fecha_hecho

In [8]:
# Filtrar y mostrar los valores nulos
threat_crimes_[threat_crimes_['fecha_hecho'].isnull()]

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,cantidad
388566,68276000,SANTANDER,Floridablanca,NaN,FEMENINO,ADULTOS,1
388567,68276000,SANTANDER,Floridablanca,NaN,FEMENINO,ADULTOS,1
388568,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,2
388569,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,1
388570,68276000,SANTANDER,Floridablanca,NaN,FEMENINO,ADULTOS,1
388571,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,1
388572,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,1
388573,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,1
388574,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,1
388575,68276000,SANTANDER,Floridablanca,NaN,MASCULINO,ADULTOS,2


Nota: Todos los valores nulos de fecha_hecho se ubican consecutivamente en el df original, indices 388559-388568 y corresponde al Florida Blanca Santader.

Dado que las fechas de los eventos tienden a ser consecutivas y no varían demasiado, optamos por la imputacion Forward-fill (imputación con el valor anterior)

In [9]:
# Imputacion con Forward-fill
threat_crimes_['fecha_hecho'] = threat_crimes_['fecha_hecho'].ffill()

In [10]:
# Verificar imputación
threat_crimes_['fecha_hecho'] = threat_crimes_['fecha_hecho'].astype(int)
threat_crimes_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
genero,0
grupo_etario,1282
cantidad,0


Nota: Los nulos grupo_etario se imputan más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [11]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['genero', 'grupo_etario', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(threat_crimes_[column].unique())
    print()


Categorías en la columna 'genero':
['MASCULINO' 'FEMENINO' 'NO REPORTA' 'NO REPORTADO' '-' 'NO APLICA']

Categorías en la columna 'grupo_etario':
['ADULTOS' 'ADOLESCENTES' 'MENORES' 'NO REPORTADO' 'ADOLECENTES' nan '-'
 'NO REPORTA' 'NO APLICA']

Categorías en la columna 'departamento':
['ANTIOQUIA' 'BOLÍVAR' 'HUILA' 'NARIÑO' 'CALDAS' 'CESAR' 'CUNDINAMARCA'
 'SUCRE' 'VALLE' 'BOYACÁ' 'NORTE DE SANTANDER' 'SANTANDER' 'TOLIMA'
 'ARAUCA' 'ATLÁNTICO' 'MAGDALENA' 'META' 'PUTUMAYO' 'QUINDÍO' 'RISARALDA'
 'VICHADA' 'CHOCÓ' 'GUAJIRA' 'AMAZONAS' 'CAQUETÁ' 'CÓRDOBA' 'CASANARE'
 'CAUCA' 'GUAVIARE' 'VAUPÉS' 'GUAINÍA' 'SAN ANDRÉS']

Categorías en la columna 'municipio':
['MEDELLÍN (CT)' 'CARTAGENA (CT)' 'PITALITO' ... 'Sácama'
 'San José del Palmar' 'Encino']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [12]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [13]:
# Aplicar la función a todas las columnas categóricas
threat_crimes_[categorical_col] = threat_crimes_[categorical_col].astype(str)
for col in categorical_col:
    threat_crimes_[col] = threat_crimes_[col].apply(remove_accents_and_special_chars)

- Mejorar consistencia de las columnas 'genero' y 'grupo_etario'

In [14]:
threat_crimes_['genero'] = threat_crimes_['genero'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    '-': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

In [15]:
threat_crimes_['grupo_etario'] = threat_crimes_['grupo_etario'].replace({
    'NO REPORTA': 'NO REPORTADO',
    'NO APLICA': 'NO REPORTADO',
    'NAN': 'NO REPORTADO',
    'NO REPORTADO': 'NO REPORTADO'
})

- Codificación de algunas variables categóricas

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [16]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df threat_crimes_

In [17]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
threat_crimes_['codigo_dane'] = threat_crimes_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = threat_crimes_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [18]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [ 7  8 12]


In [19]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_dígitos_codigo_dane,count,percentage
0,8,366055,0.887159
1,7,46559,0.112839
2,12,1,0.000002


In [20]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = threat_crimes_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
1    13001000
2    41551000
3    52001000
4    13001000
5    17001000

Muestra de registros con longitud 7:
   codigo_dane
0      5001000
29     5001000
32     8001000
33     8421000
34     8421000

Muestra de registros con longitud 12:
         codigo_dane
135506  NO REPORTADO



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 88,7% tiene 8 dígitos (al parecer se le adicionaron 3 ceros al final del código que realmente es de 5 dígitos), el 11,3% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió). En un registro en vez del código aparacece la palabra NO REPORTADO

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [21]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = threat_crimes_[threat_crimes_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [22]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
threat_crimes_['codigo_dane'] = threat_crimes_['codigo_dane'].apply(add_zero_if_length_7)

In [23]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(threat_crimes_[threat_crimes_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [24]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
threat_crimes_['codigo_dane'] = threat_crimes_['codigo_dane'].apply(trim_last_3_if_length_8)

In [25]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(threat_crimes_[threat_crimes_['codigo_dane'].str.len() == 8])

0

### Solucionar los codigo_dane que en el dataset fueron digitados erroneamente como NO REPORTADO

In [26]:
# Mostrar los dos registros donde el codigo_dane esta errado
threat_crimes_[threat_crimes_['codigo_dane'].isin(['NO REPORTADO'])]

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,cantidad
135506,NO REPORTADO,GUAINIA,NO REPORTADO,2017,FEMENINO,ADULTOS,1


Nota: Dado que no existe posibilidad para conocer el municipio donde ocurrio el hecho, procedemos a eliminar el registro

In [ ]:
# Borrar registros
threat_crimes_ = threat_crimes_[~(threat_crimes_['codigo_dane'].isin(['NO REPORTADO']))]

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(threat_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en threat_crimes_", "Departamentos dept_mpios_codes")

Departamentos en threat_crimes_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'SAN ANDRES', 'GUAJIRA'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe threat_crimes_
threat_crimes_['departamento'] = threat_crimes_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(threat_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en threat_crimes_", "Departamentos dept_mpios_codes")

Departamentos en threat_crimes_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(threat_crimes_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en threat_crimes_", "Códigos de municipios dept_mpios_codes")

Departamentos en threat_crimes_ que no están en Departamentos dept_mpios_codes:
{'52000'}


  - Identifcar los registros bajo este código que no corresponde a ningun código real de municipio

In [ ]:
threat_crimes_[threat_crimes_['codigo_dane'] == '52000']

,codigo_dane,departamento,municipio,fecha_hecho,genero,grupo_etario,cantidad
354848,52000,NARINO,-,2023,FEMENINO,ADULTOS,1
354849,52000,NARINO,-,2023,FEMENINO,ADULTOS,3
384602,52000,NARINO,NO REPORTADO,2023,MASCULINO,ADULTOS,1
384603,52000,NARINO,NO REPORTADO,2023,FEMENINO,ADULTOS,1
384604,52000,NARINO,NO REPORTADO,2023,MASCULINO,ADULTOS,1
408563,52000,NARINO,-,2024,MASCULINO,ADULTOS,1
408564,52000,NARINO,-,2024,MASCULINO,ADULTOS,1


Son 7 municipios del departamento de Narino que no estan identificados, procedemos a eliminar los registros del dataset

In [ ]:
# Eliminar filas donde 'codigo_dane' es igual a '52000'
threat_crimes_ = threat_crimes_[threat_crimes_['codigo_dane'] != '52000']

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = [ 'departamento', 'municipio']
final_threat_crimes = threat_crimes_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_threat_crimes['source_id'] = 12

In [ ]:
final_threat_crimes.columns

Index(['codigo_dane', 'fecha_hecho', 'genero', 'grupo_etario', 'cantidad',
       'source_id'],
      dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'genero': 'sex',
    'grupo_etario': 'age_group',
    'cantidad': 'amount',
    'codigo_dane': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_threat_crimes.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_threat_crimes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412607 entries, 0 to 412614
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dane_code         412607 non-null  object
 1   year_of_incident  412607 non-null  int64 
 2   sex               412607 non-null  object
 3   age_group         412607 non-null  object
 4   amount            412607 non-null  int64 
 5   source_id         412607 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 22.0+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_threat_crimes.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/threat_crimes.csv', index=False)